In [1]:
import pandas as pd
import numpy as np
import scipy
import torch
from sklearn.model_selection import KFold, cross_val_score

In [2]:
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [3]:
smiles_df = pd.read_csv('/home/sgarkot/code/attention/test_dataset.csv')
smiles_df = smiles_df['Ligand']
smiles_df

0    [H]/N=C(/c1ccc(cn1)CNC(=O)[C@@H]2C=CCN2C(=O)[C...
1    CCOC(=O)[C@@H](Cc1ccc2ccc(cc2c1)C(N)N)c3ccc(OC...
2    CC1=CC(=CC(=C1)OCCNC2=CC=[NH+]C=C2)NS(=O)(=O)C...
3    NC(=N)NCCCNC(=O)[C@@H]1CS[C@H]2CC[C@@H](N[S](=...
4                                    [NH3+]C(CO)(CO)CO
5    O=C(N(CC)CC)c3cc(c1cc2c(cc1)C(C(C(=O)O)CCCCC(=...
6    NC(=N)CCCC[C@H]([C@@H]1[C@@H](O)Cc2ccc(cc12)c3...
7    COc1cccc(c1)c2ccc3[C@@H]([C@@H](O)Cc3c2)[C@@H]...
8    CC(=O)N[C@H]1[C@@H](O)O[C@H](CO)[C@@H](O)[C@@H]1O
9    O=C(NC(CCCNC(=[NH2+])\N)C(O)CCl)C2N(C(=O)C(N)C...
Name: Ligand, dtype: object

In [4]:
from simpletransformers_sofija.simpletransformers.classification.classification_model_sofija_garkosv import ClassificationModel

model_ki = ClassificationModel('roberta', './checkpoint-3270-epoch-5', num_labels=1, args={'num_train_epochs': 4,  'regression': True, 'labels_list': [], 'config':{'output_hidden_states':True,'output_attentions':True}})
forward_result = model_ki.predict(smiles_df)

preds, model_outputs, all_embedding_outputs, all_layer_hidden_states, attention, inputs_total = forward_result


/home/sgarkot/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sgarkot/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sgarkot/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sgarkot/miniconda3/lib/python3.7/site-packages/tensorflow/python/frame

In [5]:
representations = []
for dct in inputs_total:
    for el in dct:
        if el == 'input_ids':
            for vec in dct[el]:
                representations.append(vec)

print(len(representations))
print(representations[0])

10
tensor([  0, 225,  63,  44, 594,  50,  33,  39, 324,  71,  21, 264,  12, 311,
         21,  13, 300, 263,  51, 272,  39, 266,  44,  65,  22,  39,  33, 280,
         22,  39, 263,  51, 272,  39, 266,  44, 265, 262,  23, 305,  23,  13,
        322, 263,  51,  13,  51,  13,  50,   2,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1], device='cuda:0')


In [6]:
# There are 6 attention layers, collected here
attentions_list_1 = []
attentions_list_2 = []
attentions_list_3 = []
attentions_list_4 = []
attentions_list_5 = []
attentions_list_6 = []

for a in attention:
    attentions_list_1.append(a[0])
    attentions_list_2.append(a[1])
    attentions_list_3.append(a[2])
    attentions_list_4.append(a[3])
    attentions_list_5.append(a[4])
    attentions_list_6.append(a[5])
    
all_attentions_1 = torch.cat(attentions_list_1, dim=0)
all_attentions_2 = torch.cat(attentions_list_2, dim=0)
all_attentions_3 = torch.cat(attentions_list_3, dim=0)
all_attentions_4 = torch.cat(attentions_list_4, dim=0)
all_attentions_5 = torch.cat(attentions_list_5, dim=0)
all_attentions_6 = torch.cat(attentions_list_6, dim=0)


In [7]:
# Each of the layers is a tensor of shape [12, 128, 128], 12 is a number of heads, 128 x 128 is a matrix of weights for each of the symbol
all_attentions_1[1].shape

torch.Size([12, 128, 128])

In [8]:
# visualization of how tokenization is performed
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing 

tokenizer = ByteLevelBPETokenizer(
    "checkpoint-3270-epoch-5/vocab.json",
    "checkpoint-3270-epoch-5/merges.txt",
)
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)

tokenizer.enable_truncation(max_length=512)

print("Smiles 0:", smiles_df[0])
t = [  0, 225,  63,  44, 594,  50,  33,  39, 324,  71,  21, 264,  12, 311,
         21,  13, 300, 263,  51, 272,  39, 266,  44,  65,  22,  39,  33, 280,
         22,  39, 263,  51, 272,  39, 266,  44, 265, 262,  23, 305,  23,  13,
        322, 263,  51,  13,  51,  13,  50,   2,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1]
to_decode =  ['Ġ', '[', 'H', ']/', 'N', '=', 'C', '(/', 'c', '1', 'ccc', '(', 'cn', '1', ')', 'CNC', '(=', 'O', ')[', 'C', '@@', 'H', ']', '2', 'C', '=', 'CCN', '2', 'C', '(=', 'O', ')[', 'C', '@@', 'H', '](', 'CC', '3', 'CCCCC', '3', ')', 'NCC', '(=', 'O', ')', 'O', ')', 'N']
tokenizer.decode(t)

Smiles 0: [H]/N=C(/c1ccc(cn1)CNC(=O)[C@@H]2C=CCN2C(=O)[C@@H](CC3CCCCC3)NCC(=O)O)N


'<s> [H]/N=C(/c1ccc(cn1)CNC(=O)[C@@H]2C=CCN2C(=O)[C@@H](CC3CCCCC3)NCC(=O)O)N</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [9]:
from paint_attention99 import Molecule_Attention 
for i in range(10):
    smiles = smiles_df[i] 
    embedding = representations[i] 
    attention_matrix = all_attentions_2[i]
    attention_layer = 2
    mol_att = Molecule_Attention( smiles, embedding, attention_matrix, attention_layer, i )
    for j in range(len(attention_matrix)):
        mol_att.visualize(j)

INFO:rdkit:Enabling RDKit 2020.03.1 jupyter extensions


Part of the molecule: CCCCC 
Weight of attention paid to it: 0.4388352334499359
None patt </s>
substring not found
Part of the molecule: </s> 
Weight of attention paid to it: 0.08500739187002182
None patt <s>
substring not found
Part of the molecule: <s> 
Weight of attention paid to it: 0.04342831298708916
None patt  
substring not found
Part of the molecule:   
Weight of attention paid to it: 0.00010652905621100217
None patt [
Part of the molecule: [ 
Weight of attention paid to it: 0.0015990609535947442
None patt H
Still none patt:
 [H
Part of the molecule: H 
Weight of attention paid to it: 0.008658748120069504
None patt ]/
Still none patt:
 H]
Part of the molecule: ]/ 
Weight of attention paid to it: 0.017878994345664978
Part of the molecule: N 
Weight of attention paid to it: 0.007857653312385082
None patt =
Still none patt:
 N=
Part of the molecule: = 
Weight of attention paid to it: 0.01265972014516592
Part of the molecule: C 
Weight of attention paid to it: 0.009758379310369492

RDKit ERROR: [11:32:08] SMARTS Parse Error: syntax error while parsing: </s>
RDKit ERROR: [11:32:08] SMARTS Parse Error: Failed parsing SMARTS '</s>' for input: '</s>'
RDKit ERROR: [11:32:08] SMARTS Parse Error: syntax error while parsing: <s>
RDKit ERROR: [11:32:08] SMARTS Parse Error: Failed parsing SMARTS '<s>' for input: '<s>'
RDKit ERROR: [11:32:08] SMARTS Parse Error: syntax error while parsing: 
RDKit ERROR: [11:32:08] SMARTS Parse Error: Failed parsing SMARTS ' ' for input: ' '
RDKit ERROR: [11:32:08] SMARTS Parse Error: syntax error while parsing: [
RDKit ERROR: [11:32:08] SMARTS Parse Error: Failed parsing SMARTS '[' for input: '['
RDKit ERROR: [11:32:08] SMARTS Parse Error: syntax error while parsing: H
RDKit ERROR: [11:32:08] SMARTS Parse Error: Failed parsing SMARTS 'H' for input: 'H'
RDKit ERROR: [11:32:08] SMARTS Parse Error: syntax error while parsing: [H
RDKit ERROR: [11:32:08] SMARTS Parse Error: Failed parsing SMARTS '[H' for input: '[H'
RDKit ERROR: [11:32:08] SMART

None patt 1
Still none patt:
 c1
Part of the molecule: 1 
Weight of attention paid to it: 0.008497452363371849
Part of the molecule: ccc 
Weight of attention paid to it: 0.029889466241002083
Part of the molecule: cn 
Weight of attention paid to it: 0.00759155722334981
None patt 1
Still none patt:
 c1
Part of the molecule: 1 
Weight of attention paid to it: 0.0029209679923951626
Part of the molecule: CNC 
Weight of attention paid to it: 0.014493131078779697
None patt (=
Still none patt:
 C(
Part of the molecule: (= 
Weight of attention paid to it: 0.009528026916086674
Part of the molecule: O 
Weight of attention paid to it: 0.008676670491695404
None patt )[
Still none patt:
 O)
Part of the molecule: )[ 
Weight of attention paid to it: 0.008071333169937134
Part of the molecule: C 
Weight of attention paid to it: 0.0076307873241603374
None patt @@
Still none patt:
 C@
Part of the molecule: @@ 
Weight of attention paid to it: 0.008044915273785591
None patt H
Still none patt:
 [H
Part of th

RDKit ERROR: [11:32:08] SMARTS Parse Error: syntax error while parsing: )[
RDKit ERROR: [11:32:08] SMARTS Parse Error: Failed parsing SMARTS ')[' for input: ')['
RDKit ERROR: [11:32:08] SMARTS Parse Error: syntax error while parsing: O)
RDKit ERROR: [11:32:08] SMARTS Parse Error: Failed parsing SMARTS 'O)' for input: 'O)'
RDKit ERROR: [11:32:08] SMARTS Parse Error: syntax error while parsing: @@
RDKit ERROR: [11:32:08] SMARTS Parse Error: Failed parsing SMARTS '@@' for input: '@@'
RDKit ERROR: [11:32:08] SMARTS Parse Error: syntax error while parsing: C@
RDKit ERROR: [11:32:08] SMARTS Parse Error: Failed parsing SMARTS 'C@' for input: 'C@'
RDKit ERROR: [11:32:08] SMARTS Parse Error: syntax error while parsing: H
RDKit ERROR: [11:32:08] SMARTS Parse Error: Failed parsing SMARTS 'H' for input: 'H'
RDKit ERROR: [11:32:08] SMARTS Parse Error: syntax error while parsing: [H
RDKit ERROR: [11:32:08] SMARTS Parse Error: Failed parsing SMARTS '[H' for input: '[H'
RDKit ERROR: [11:32:08] SMARTS 

...

None patt )[
Still none patt:
 O)
Part of the molecule: )[ 
Weight of attention paid to it: 0.01522491592913866
Part of the molecule: C 
Weight of attention paid to it: 0.004699354525655508
None patt @@
Still none patt:
 C@
Part of the molecule: @@ 
Weight of attention paid to it: 0.03942393139004707
None patt H
Still none patt:
 @H
Part of the molecule: H 
Weight of attention paid to it: 0.04058024659752846
None patt ]
Still none patt:
 H]
Part of the molecule: ] 
Weight of attention paid to it: 0.012910575605928898
None patt 1
Still none patt:
 ]1
Part of the molecule: 1 
Weight of attention paid to it: 0.01024231594055891
Part of the molecule: O 
Weight of attention paid to it: 0.014316821470856667
None patt </s>
substring not found
Part of the molecule: </s> 
Weight of attention paid to it: 0.0037006910424679518
None patt  
substring not found
Part of the molecule:   
Weight of attention paid to it: 0.07351633906364441
None patt [
Still did not catch a pattern from SMILES:
 N[
Part

RDKit ERROR: [11:32:59] SMARTS Parse Error: syntax error while parsing: <s>
RDKit ERROR: [11:32:59] SMARTS Parse Error: Failed parsing SMARTS '<s>' for input: '<s>'
RDKit ERROR: [11:32:59] SMARTS Parse Error: syntax error while parsing: 1
RDKit ERROR: [11:32:59] SMARTS Parse Error: Failed parsing SMARTS '1' for input: '1'
RDKit ERROR: [11:32:59] SMARTS Parse Error: unclosed ring for input: 'c1'
RDKit ERROR: [11:32:59] SMARTS Parse Error: syntax error while parsing: </s>
RDKit ERROR: [11:32:59] SMARTS Parse Error: Failed parsing SMARTS '</s>' for input: '</s>'
RDKit ERROR: [11:32:59] SMARTS Parse Error: syntax error while parsing: 
RDKit ERROR: [11:32:59] SMARTS Parse Error: Failed parsing SMARTS ' ' for input: ' '
RDKit ERROR: [11:32:59] SMARTS Parse Error: syntax error while parsing: =
RDKit ERROR: [11:32:59] SMARTS Parse Error: Failed parsing SMARTS '=' for input: '='
RDKit ERROR: [11:32:59] SMARTS Parse Error: syntax error while parsing: O=
RDKit ERROR: [11:32:59] SMARTS Parse Error

None patt <s>
substring not found
Part of the molecule: <s> 
Weight of attention paid to it: 0.05361930653452873
None patt 1
Still did not catch a pattern from SMILES:
 c1
Part of the molecule: 1 
Weight of attention paid to it: 0.1480734497308731
Part of the molecule: CCC 
Weight of attention paid to it: 0.21678493916988373
None patt </s>
substring not found
Part of the molecule: </s> 
Weight of attention paid to it: 0.21394848823547363
None patt  
substring not found
Part of the molecule:   
Weight of attention paid to it: 0.00010254199150949717
Part of the molecule: O 
Weight of attention paid to it: 0.005003265105187893
None patt =
Still none patt:
 O=
Part of the molecule: = 
Weight of attention paid to it: 0.008265352807939053
Part of the molecule: C 
Weight of attention paid to it: 0.004336507525295019
None patt (
Still none patt:
 C(
Part of the molecule: ( 
Weight of attention paid to it: 0.007952969521284103
Part of the molecule: NC 
Weight of attention paid to it: 0.01471695

Part of the molecule: N 
Weight of attention paid to it: 0.007945036515593529
None patt (
Still none patt:
 C(
Part of the molecule: ( 
Weight of attention paid to it: 0.008723060600459576
Part of the molecule: C 
Weight of attention paid to it: 0.007299668621271849
None patt (=
Still none patt:
 C(
Part of the molecule: (= 
Weight of attention paid to it: 0.008058559149503708
Part of the molecule: O 
Weight of attention paid to it: 0.007820656523108482
None patt )
Still none patt:
 ])
Part of the molecule: ) 
Weight of attention paid to it: 0.007808685302734375
Part of the molecule: C 
Weight of attention paid to it: 0.007835986092686653
None patt (
Still none patt:
 C(
Part of the molecule: ( 
Weight of attention paid to it: 0.010746141895651817
Part of the molecule: N 
Weight of attention paid to it: 0.006515572313219309
None patt )
Still none patt:
 ])
Part of the molecule: ) 
Weight of attention paid to it: 0.008365001529455185
Part of the molecule: Cc 
Weight of attention paid to

RDKit ERROR: [11:33:00] SMARTS Parse Error: syntax error while parsing: (
RDKit ERROR: [11:33:00] SMARTS Parse Error: Failed parsing SMARTS '(' for input: '('
RDKit ERROR: [11:33:00] SMARTS Parse Error: syntax error while parsing: C(
RDKit ERROR: [11:33:00] SMARTS Parse Error: Failed parsing SMARTS 'C(' for input: 'C('
RDKit ERROR: [11:33:00] SMARTS Parse Error: syntax error while parsing: (=
RDKit ERROR: [11:33:00] SMARTS Parse Error: Failed parsing SMARTS '(=' for input: '(='
RDKit ERROR: [11:33:00] SMARTS Parse Error: syntax error while parsing: C(
RDKit ERROR: [11:33:00] SMARTS Parse Error: Failed parsing SMARTS 'C(' for input: 'C('
RDKit ERROR: [11:33:00] SMARTS Parse Error: syntax error while parsing: )
RDKit ERROR: [11:33:00] SMARTS Parse Error: Failed parsing SMARTS ')' for input: ')'
RDKit ERROR: [11:33:00] SMARTS Parse Error: syntax error while parsing: ])
RDKit ERROR: [11:33:00] SMARTS Parse Error: Failed parsing SMARTS '])' for input: '])'
RDKit ERROR: [11:33:00] SMARTS Par